## Analysis
- there are 2 hotel - City Hotel and Resort Hotel
- August has the highest booking and January has the least bookings
- there are people from 177 countries that did the booking
- the best room w.r.t City hotel is 'G' and Resort Hotel is 'H'
- 2016 and 2017 in City and Resort hotel we have the maximum price in August and November is the cheapest
- In 2015 the highest in City and Resort hotel was in September and August.
- Resort hotel has more variabel data as compared to City hotel w.r.t. price.
- City hotel is much populated than Resort Hotel 
- There is no columns with zero variance
- Important Columns :
* 'lead_time', 'stays_in_week_nights', 'adults', 'previous_cancellations',
       'booking_changes', 'agent', 'days_in_waiting_list', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests', 'hotel',
       'country', 'market_segment', 'distribution_channel',
       'assigned_room_type', 'deposit_type', 'customer_type',
       'reservation_status', 'month', 'date'
- Naive Bayes took the less time and accuracy is almost 1

## Things to do-
- there are 5 types of meals BB, HB, SC, Undefined, FB                           ------------------handle this Undefined
- missing values ------- children  country  agent  company                         -----------------handle the missing values
* country is the only categorical data that contain missing values others are numerical
- there are some fields where the entry is there but there are no ADULTS ,CHILDREN or BABIES    ----------handle them 
- convert the reservation_status_date to datetime from object
- is_canceled is the feature that should be categorical as its the predictor
- check out for outliers
- make model and check with cross validation 
- models to make random forest, naive bayes, logistic regression, KNN 

### EDA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)

In [ ]:
df= pd.read_csv('../input/hotel-booking-demand/hotel_bookings.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()


In [ ]:
### dropping the column company
df.drop('company',axis=1,inplace=True)

In [ ]:
sns.histplot(x='agent',data=df,kde=True)

In [ ]:
median = df.agent.median()
df.agent.fillna(median,inplace=True)

In [ ]:
sns.histplot(x='agent',data=df,kde=True)

In [ ]:
median = df.children.median()
df.children.fillna(median,inplace=True)

In [ ]:
df.country.isnull().sum()/df.country.count()      #### as the data is less than 40 % we can use the mode to impute the NAN values

In [ ]:
mode = df.country.mode()[0]
df.country.fillna(mode,inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
cat = [cat for cat in df if(df[cat].dtype =='O')]
print(len(cat))
cat

In [ ]:
num = [num for num in df if(df[num].dtype !='O')]
print(len(num))
num

In [ ]:
for i in cat:
    print(df[i].value_counts())

In [ ]:
l=['adults','children','babies']

for i in l:
    print(i," ",df[i].unique())

In [ ]:
filter = (df.adults ==0) & (df.children ==0) & (df.babies ==0) 
df[filter]

In [ ]:
df_new = df[~filter]

In [ ]:
df_new.country.unique()

In [ ]:
df_not_cancelled = df[df['is_canceled']==0]
df_cancelled = df[df['is_canceled']==1]

In [ ]:
df_not_cancelled['country'].value_counts()

In [ ]:
df_not_cancelled['country'].value_counts().plot.barh(figsize=(10,10))

In [ ]:
df_new.columns

In [ ]:
sns.set_theme()
plt.figure(figsize=(20,10))
sns.boxplot(x='reserved_room_type',y='adr',hue='hotel',data=df_not_cancelled)
plt.xlabel('Rooms Reserved')
plt.ylabel('Price')


In [ ]:
df_not_cancelled.arrival_date_year.value_counts().plot.bar(figsize=(10,10))

In [ ]:
df_2015 = df_not_cancelled[df_not_cancelled['arrival_date_year']==2015]
df_2016 = df_not_cancelled[df_not_cancelled['arrival_date_year']==2016]
df_2017 = df_not_cancelled[df_not_cancelled['arrival_date_year']==2017]

In [ ]:
df_2015.groupby(['hotel','arrival_date_month'])['adr'].mean().plot.barh(figsize=(10,10))

In [ ]:
df_2016.groupby(['hotel','arrival_date_month'])['adr'].mean().plot.barh(figsize=(10,10))

In [ ]:
df_2017.groupby(['hotel','arrival_date_month'])['adr'].mean().plot.barh(figsize=(10,10))

In [ ]:
df_2017.arrival_date_month.unique()

In [ ]:
plt.figure(figsize=(10,10))
sns.lineplot(x='arrival_date_month',y='adr',data=df_new,hue='hotel')
plt.xticks(rotation=90)

In [ ]:
plt.figure(figsize=(10,10))
plt.xticks(rotation=90)
sns.lineplot(x='arrival_date_month',y='adr',data=df_2015,hue='hotel')

In [ ]:
plt.figure(figsize=(10,10))
plt.xticks(rotation=90)
sns.lineplot(x='arrival_date_month',y='adr',data=df_2016,hue='hotel')

In [ ]:
plt.figure(figsize=(10,10))
plt.xticks(rotation=90)
sns.lineplot(x='arrival_date_month',y='adr',data=df_2017,hue='hotel')

In [ ]:
df_guest_resort = df_new[df_new['hotel']=='Resort Hotel']
resort = df_guest_resort['arrival_date_month'].value_counts()

In [ ]:
df_guest_city = df_new[df_new['hotel']=='City Hotel']
city = df_guest_city['arrival_date_month'].value_counts()

In [ ]:
df_guests_combined = pd.concat([resort,city],axis=1)

In [ ]:
df_guests_combined.columns = ['Resort','City']
df_guests_combined

In [ ]:
df_guests_combined['Months'] = df_guests_combined.index

In [ ]:
df_guests_combined.sort_values(by='Months')

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(df_guests_combined.Resort,label='resort')
plt.plot(df_guests_combined.City,label='city')
plt.ylabel('Customers in number')
plt.xlabel('Months')
plt.legend(title='Hotel')
plt.xticks(rotation=90)

In [ ]:
df_new.head()

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df_new.corr(),annot=True)

In [ ]:
y = df_new['is_canceled']

In [ ]:
df_new = df_new.drop('is_canceled',axis=1)

In [ ]:
df_new = pd.concat([df_new,y],axis=1)

In [ ]:
df_categorical = df_new[cat]

In [ ]:
df_categorical.head()

In [ ]:
df_categorical['reservation_status_date']= pd.to_datetime(df_categorical['reservation_status_date'])

In [ ]:
df_categorical['year']= df_categorical['reservation_status_date'].dt.year
df_categorical['month']= df_categorical['reservation_status_date'].dt.month
df_categorical['date']= df_categorical['reservation_status_date'].dt.date

In [ ]:
df_categorical.drop(['reservation_status_date'],axis=1,inplace=True)

In [ ]:
df_categorical['cancelled'] = df['is_canceled']

In [ ]:
df_categorical.head()

In [ ]:
for i in df_categorical.columns:
    dict_convert = df_categorical.groupby([i])['cancelled'].mean().to_dict()
    df_categorical[i] = df_categorical[i].map(dict_convert)

In [ ]:
df

In [ ]:
ncn = pd.concat([df_new[num],df_categorical],axis=1)

In [ ]:
df_new[num].shape,df_categorical.shape

In [ ]:
df_new.shape

In [ ]:
ncn.drop('is_canceled',axis=1,inplace=True)

In [ ]:
ncn.isnull().sum()

In [ ]:
def trans(col):
    plt.figure(figsize=(5,5))
    sns.distplot(ncn[col])
    plt.xlabel(col)
    plt.title(col)
    plt.show()
    

In [ ]:
# for col in nc.columns:
#     trans(col)

In [ ]:
# def trans(col):
#     plt.figure(figsize=(8,8))
#     sns.boxplot(ncn[col])
#     plt.xlabel(col)
#     plt.title(col)
#     plt.show()
    
# for col in ncn.columns:
#     trans(col)
    
    
    
#     lead time, Stay_in_weekend_nights, stay_in_week_night, agent, adr, total_of_special_request


In [ ]:
def trans(col):
    plt.figure(figsize=(10,5))
    plt.subplot(1, 2, 1)
    sns.boxplot(ncn[col])
    plt.xlabel(col)
    plt.title(col)
    plt.subplot(1, 2, 2)
    sns.boxplot(np.log1p(ncn[col]))
    plt.xlabel(col)
    plt.title(col)
    plt.show()
    
for col in ['lead_time','agent', 'adr', 'total_of_special_requests']:
    trans(col)

In [ ]:
sns.boxplot(ncn.adr**(1/2))

In [ ]:
import scipy.stats as stats
import pylab

def trans(col):
    plt.figure(figsize=(10,5))
    plt.subplot(1, 2, 1)
    sns.distplot(np.log1p(ncn[col]))
    plt.xlabel(col)
    plt.title(col)
    plt.subplot(1, 2, 2)
    stats.probplot(np.log1p(ncn[col]),dist='norm',plot=pylab)
    plt.xlabel(col)
    plt.title(col)
    plt.show()
    
for col in ['lead_time','agent', 'adr']:
    trans(col)

In [ ]:
def trans(col):
    plt.figure(figsize=(10,5))
    plt.subplot(1, 2, 1)
    sns.distplot(np.log1p(ncn[col]))
    plt.xlabel(col)
    plt.title(col)
    plt.subplot(1, 2, 2)
    sns.boxplot(np.log1p(ncn[col]))
    plt.xlabel(col)
    plt.title(col)
    plt.show()
    
for col in ['lead_time','agent','stays_in_weekend_nights']:
    trans(col)

In [ ]:
ncn['lead_time'] = np.log1p(ncn['lead_time'])
ncn['agent'] = np.log1p(ncn['agent'])
ncn['stays_in_weekend_nights'] = np.log1p(ncn['stays_in_weekend_nights'])

In [ ]:
#### applying IQR  at adr

Q1 = ncn['adr'].quantile(0.25)
Q3 = ncn['adr'].quantile(0.75)
IQR = Q3-Q1

lower = ncn['adr'].quantile(0.25)-(1.5*IQR)
upper = ncn['adr'].quantile(0.75)+(1.5*IQR)

In [ ]:
lower,upper

In [ ]:
### adr transfrom

data = ncn.copy() 
data.loc[data['adr']>210.75,'adr'] = 210.75
data.loc[data['adr']<0,'adr'] = 0

### total_of_special_requests
data['total_of_special_requests'] = data['total_of_special_requests']**(1/2)



In [ ]:
sns.boxplot(data['adr'])

In [ ]:
def trans(col):
    plt.figure(figsize=(10,5))
    plt.subplot(1, 2, 1)
    sns.distplot(data[col])
    plt.xlabel(col)
    plt.title(col)
    plt.subplot(1, 2, 2)
    sns.boxplot(data[col])
    plt.xlabel(col)
    plt.title(col)
    plt.show()
    
for col in ['lead_time','agent','adr','stays_in_weekend_nights']:
    trans(col)

In [ ]:
data.head()

In [ ]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=15)

In [ ]:
from sklearn.feature_selection import mutual_info_classif
mi = mutual_info_classif(X_train,y_train)
mi

In [ ]:
mi = pd.Series(mi)

In [ ]:
mi.index = X_train.columns

In [ ]:
mi.sort_values(ascending=True).plot.barh(figsize=(10,10))

In [ ]:
from sklearn.feature_selection import SelectKBest

top_20 = SelectKBest(mutual_info_classif,k=20)
top_20.fit(X_train,y_train)
X_train.columns[top_20.get_support()]

In [ ]:
selected_features= data[X_train.columns[top_20.get_support()]]

In [ ]:
len(selected_features.columns)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(selected_features,y,test_size=0.3,random_state=15)

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
lg=LogisticRegression()
lg.fit(X_train,y_train)
y_pred = lg.predict(X_test)
print('Confusion Metric:\n\n ',  confusion_matrix(y_test,y_pred))
print('\n\n Accuracy Metric:\n ',  accuracy_score(y_test,y_pred))
print('\n\nClassification Metric:\n ',  classification_report(y_test,y_pred))

In [ ]:
data.cancelled.value_counts()  ############# almosts same value so Kfold will work 

In [ ]:
from sklearn.model_selection import cross_val_score
val = cross_val_score(lg,selected_features,y,cv=10)

In [ ]:
val.mean()  #### this the cross-validated accuracy

## Model Implementation

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
models=[]

models.append(('Naive Bayes',GaussianNB()))
models.append(('RandomForest Classifier ',RandomForestClassifier()))
models.append(('KNeighbors Classifier',KNeighborsClassifier()))

In [ ]:
for name,model in models:
    print(f"Model: {name}")
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print('Confusion Metric:\n\n ',  confusion_matrix(y_test,y_pred))
    print('\n\n Accuracy Metric:\n ',  accuracy_score(y_test,y_pred))
    print('\n\nClassification Metric:\n ',  classification_report(y_test,y_pred))
          

In [ ]:
%%time
for name,model in models:
    print(f"Model: {name}")
    val = cross_val_score(model,X,y,cv=3)
    print('Accuracy mean:\n\n ',val.mean())    

In [ ]:
#### Naive Bayes took less time and wins the race